송하인과 수학인의 5개 숫자는 곧 지역을 의미한다.

[GIS]이용해보기

물픔_카테고리와 지역을 잡으면 지역별로 물품을 볼 수 있다

운송장_건수와 지역을 잡으면 지역별로 교류량을 파악할 수 있다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches

# seaborn setting
sns.set_theme(style='whitegrid')
sns.set_palette("twilight")

In [2]:
# 구글 colab 환경일 경우 그래프에 한글 폰트 사용을 위한 설정
import matplotlib.font_manager as mfm
path = 'noviceml/font/NanumBarunGothic.ttf'
fontprop = mfm.FontProperties(fname=path, size=18)


**데이터 불러오기**

In [3]:
sample = pd.read_csv('/content/sample_submission.csv')
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')

**Train set**

In [4]:
train.shape
train

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3
...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,스포츠잡화,3
31680,31680,1129000014045300,5011000319087100,스마트디바이스,4
31681,31681,1129000014045300,5011000263065200,스마트디바이스,6
31682,31682,4127300065073100,5011000264061200,지갑,7


In [10]:
# 전체 열 이름 입력하기
train.columns = ['index',	'송하인_격자공간고유번호',	'수하인_격자공간고유번호',	'물품_카테고리'	,'운송장_건수']

# 선택하여 열 이름 변경하기
train.rename(columns={'index':'index','송하인_격자공간고유번호':'Send','수하인_격자공간고유번호':'Rec','물품_카테고리':'item', '운송장_건수':'counts'},inplace=True)

In [11]:
# 전체 열 이름 입력하기
test.columns = ['index',	'송하인_격자공간고유번호',	'수하인_격자공간고유번호',	'물품_카테고리']

# 선택하여 열 이름 변경하기
test.rename(columns={'index':'index','송하인_격자공간고유번호':'Send','수하인_격자공간고유번호':'Rec','물품_카테고리':'item'},inplace=True)

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   31684 non-null  int64 
 1   Send    31684 non-null  int64 
 2   Rec     31684 non-null  int64 
 3   item    31684 non-null  object
 4   counts  31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


In [13]:

train['Send_grid']=train['Send'].astype(str).apply(lambda x: x[:5]).astype(int)
train['Rec_grid']=train['Rec'].astype(str).apply(lambda x: x[:5]).astype(int)
test['Send_grid']=test['Send'].astype(str).apply(lambda x: x[:5]).astype(int)
test['Rec_grid']=test['Rec'].astype(str).apply(lambda x: x[:5]).astype(int)

In [14]:
train 

,index,Send,Rec,item,counts,Send_grid,Rec_grid
0,0,5011000595017300,2871000192069300,음반,3,50110,28710
1,1,4148000690043300,5011000264024400,문화컨텐츠,3,41480,50110
2,2,5011000078068400,1120000007005400,농산물,3,50110,11200
3,3,4127100048006400,5011000587019400,기타식품,7,41271,50110
4,4,5011000078068400,2823700010076300,농산물,3,50110,28237
...,...,...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,스포츠잡화,3,44710,50110
31680,31680,1129000014045300,5011000319087100,스마트디바이스,4,11290,50110
31681,31681,1129000014045300,5011000263065200,스마트디바이스,6,11290,50110
31682,31682,4127300065073100,5011000264061200,지갑,7,41273,50110


In [16]:
import os

f_path = './'
f_name = "eng_train.csv"

train.to_csv(os.path.join(f_path, f_name), index=False)


In [17]:
eng = pd.read_csv('/content/eng_train.csv')
eng.head()

,index,Send,Rec,item,counts,Send_grid,Rec_grid
0,0,5011000595017300,2871000192069300,음반,3,50110,28710
1,1,4148000690043300,5011000264024400,문화컨텐츠,3,41480,50110
2,2,5011000078068400,1120000007005400,농산물,3,50110,11200
3,3,4127100048006400,5011000587019400,기타식품,7,41271,50110
4,4,5011000078068400,2823700010076300,농산물,3,50110,28237


In [15]:
test

,index,Send,Rec,item,Send_grid,Rec_grid
0,0,4167000577042200,5011000435014100,선케어,41670,50110
1,1,1156000009012200,5011000172034400,구강위생용품,11560,50110
2,2,4122000363057300,5011000361097300,캠핑,41220,50110
3,3,5011000436041400,2826000084036400,아웃도어가구,50110,28260
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식,41500,50110
...,...,...,...,...,...,...
7915,7915,5011000266051200,4623000417038100,농산물,50110,46230
7916,7916,1154500001098300,5011000264055100,문화컨텐츠,11545,50110
7917,7917,5013000610049100,1147000018091400,농산물,50130,11470
7918,7918,5013000610049100,3117000039026100,농산물,50130,31170


In [18]:
import os

f_path = './'
f_name = "eng_test.csv"

test.to_csv(os.path.join(f_path, f_name), index=False)

In [19]:
test.head()

,index,Send,Rec,item,Send_grid,Rec_grid
0,0,4167000577042200,5011000435014100,선케어,41670,50110
1,1,1156000009012200,5011000172034400,구강위생용품,11560,50110
2,2,4122000363057300,5011000361097300,캠핑,41220,50110
3,3,5011000436041400,2826000084036400,아웃도어가구,50110,28260
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식,41500,50110


In [21]:
# 전체 열 이름 입력하기
sample.columns = ['index','운송장_건수']

# 선택하여 열 이름 변경하기
sample.rename(columns={'index':'index','운송장_건수':'counts'},inplace=True)

In [22]:
import os

f_path = './'
f_name = "eng_sample.csv"

sample.to_csv(os.path.join(f_path, f_name), index=False)